In [1]:
# grep -i "gw01.extranet.frachtwerk.de filterlog" pfsense.log > pfsense.csv
import pyarrow.parquet as pq
import pandas
from urllib.request import urlopen
from json import load

In [2]:
# Full parquet needs to be build
# df = pd.read_parquet('./data/df_v4.parquet', engine='pyarrow') 
df = pandas.read_parquet('./data/df_v4_10000', engine='pyarrow') 

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (17,20,21,24,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
test_df = df.head(100)

In [10]:
# add more meta data for each source ip address
def returnJsonValueIfKeyPresent(key='', data={}):
    if key in data:
        return data[key]
    else:
        return 0

def getIpInfo(addr='', freq=0):
    url = 'https://ipinfo.io/' + addr + '/json'
    res = urlopen(url)
    data = load(res)
    
    ip=returnJsonValueIfKeyPresent('ip', data)
    org=returnJsonValueIfKeyPresent('org', data)
    city=returnJsonValueIfKeyPresent('city', data)
    country=returnJsonValueIfKeyPresent('country', data)
    region=returnJsonValueIfKeyPresent('region', data)
    postal=returnJsonValueIfKeyPresent('postal', data)
    hostname=returnJsonValueIfKeyPresent('hostname', data)
    location=returnJsonValueIfKeyPresent('loc', data)
    timezone=returnJsonValueIfKeyPresent('timezone', data)
    
    print ('ip : {4} Region : {1} Country : {2} City : {3} Org : {0}'.format(org,region,country,city,ip))
    ip_meta_data = {'ip': ip, 'frequency': freq, 'org': org, 'city': city, 'country': country, 'region': region, 'postal': postal, 'hostname': hostname, 'location': location, 'timezone': timezone}
    return pandas.Series(ip_meta_data)

ip_df = pandas.DataFrame(columns=['ip', 'frequency', 'org', 'city', 'country', 'region', 'postal', 'hostname', 'location', 'timezone'])

# Get ip address information for top 1000 (frequency)
for ip, freq in df['source_ip'].value_counts()[:1000].iteritems():
    if ip != 'nan':
        ip_df = ip_df.append(getIpInfo(ip, freq), ignore_index=True)

ip : 54.39.130.200 Region : Quebec Country : CA City : Montréal Org : AS16276 OVH SAS
ip : 5.182.201.201 Region : Berlin Country : DE City : Berlin Org : AS197540 netcup GmbH
ip : 172.25.0.3 Region : 0 Country : 0 City : 0 Org : 0
ip : 95.156.250.54 Region : England Country : GB City : London Org : AS199610 marbis GmbH
ip : 213.186.33.5 Region : Île-de-France Country : FR City : Paris Org : AS16276 OVH SAS
ip : 46.251.234.89 Region : England Country : GB City : London Org : AS199610 marbis GmbH
ip : 164.90.185.34 Region : Hesse Country : DE City : Frankfurt am Main Org : AS14061 DigitalOcean, LLC
ip : 45.237.108.30 Region : Santa Catarina Country : BR City : Florianópolis Org : AS262481 NEOREDE TELECOMUNICAÇÃO EIRELI - EPP
ip : 74.91.115.69 Region : New Jersey Country : US City : Clifton Org : AS14586 Nuclearfallout Enterprises, Inc.
ip : 104.153.105.250 Region : Ohio Country : US City : Columbus Org : AS14586 Nuclearfallout Enterprises, Inc.
ip : 54.36.118.81 Region : Hauts-de-France 

HTTPError: HTTP Error 429: Too Many Requests

In [11]:
ip_df.to_csv('./ip_meta_data_source.csv', sep=',')